In [1]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import sys 
sys.path.insert(0, '../src/')

In [2]:
mass = np.load("../../data/perfect-MW-analog/acc_mass_10k.npy")
red = np.load("../../data/perfect-MW-analog/acc_redshift_10k.npy")
surv_mask = np.load("../../data/perfect-MW-analog/surv_mask_10k.npy")

In [3]:
surv_mass = np.ma.filled(np.ma.masked_array(mass, mask=~surv_mask),fill_value=np.nan)
zero_mask = np.argwhere(np.isnan(surv_mass).all(axis=1)).flatten() # to fix the dead runs
surv_mass = np.delete(surv_mass, zero_mask, axis=0)
red = np.delete(red, zero_mask, axis=0)
#np.save("../../data/perfect-MW-analog/surv_mass_10k.npy", surv_mass)

In [4]:
surv_mass = np.log10(surv_mass[:,1:]) # to get rid of the host halo mass
red = red[:,1:]
surv_mass.shape

(10510, 1999)

In [5]:
delete = surv_mass.shape[0] - 10100
delete_spill = np.random.choice(range(surv_mass.shape[0]), delete, replace=False)

surv_mass = np.delete(surv_mass, delete_spill, axis=0)
red = np.delete(red, delete_spill, axis=0)

In [6]:
Nsamp = 100
Nsaga = int(surv_mass.shape[0]/Nsamp) #dividing by the number of samples to get the number of SAGA surveys

In [7]:
SAGA_split_mass = np.split(surv_mass, Nsaga)
SAGA_split_red = np.split(red, Nsaga)

host_ids = np.arange(surv_mass.shape[0])

In [8]:
Nsub = []
M_acc = []
z_acc = []
SAGA_id = []
tree_id = []
sat_id = []

for isaga in range(Nsaga):
    for itree in range(Nsamp):
        Nsub_i = np.argwhere(~np.isnan(SAGA_split_mass[isaga][itree]))[:,0]
        for j, isat in enumerate(Nsub_i):
            Nsub.append(len(Nsub_i))
            SAGA_id.append(isaga)
            tree_id.append(itree+1)
            sat_id.append(j+1)
            M_acc.append(SAGA_split_mass[isaga][itree][isat])
            z_acc.append(SAGA_split_red[isaga][itree][isat])

In [10]:
data = Table([sat_id, tree_id, SAGA_id, Nsub, M_acc, z_acc], names=("sat_id", "tree_id", "SAGA_id", "Nsub", "M_acc", "z_acc"))
#data.write("../../data/perfect-MW-analog/101_SAGA_samples.dat")
data

sat_id,tree_id,SAGA_id,Nsub,M_acc,z_acc
int64,int64,int64,int64,float64,float64
1,1,0,238,8.31288691959396,0.017398746855025325
2,1,0,238,8.51316413193678,0.030740407989695993
3,1,0,238,8.958255957142162,0.035245056648130636
4,1,0,238,8.454606311168527,0.035245056648130636
5,1,0,238,8.533706417165762,0.05355973930933444
6,1,0,238,8.162913018984792,0.05355973930933444
7,1,0,238,8.255860458439095,0.06289961159998311
8,1,0,238,9.231466385304424,0.06289961159998311
9,1,0,238,8.608039202046559,0.07236503277627311
